<a href="https://colab.research.google.com/github/gosandayo/Exercises_for_Akari/blob/main/Exercise7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7 Ensemble learning and Random Forest


## 7.8

In [1]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False,
                                parser='auto')

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

X_test = X_mnist[:10000]
y_test = y_mnist[:10000]
X_train = X_mnist[10000:60000]
y_train = y_mnist[10000:60000]
X_valid = X_mnist[60000:]
y_valid = y_mnist[60000:]

In [ ]:


rdf_clf = RandomForestClassifier(random_state = 42, n_estimators = 100)
rdf_clf.fit(X_train, y_train)

svc_clf = LinearSVC(random_state = 42, dual = True, max_iter = 100, tol = 20)
svc_clf.fit(X_train, y_train)

extratree_clf = ExtraTreesClassifier(random_state = 42, n_estimators = 100)
extratree_clf.fit(X_train, y_train)

mlp_clf = MLPClassifier(random_state=42)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

In [ ]:
classifiers = [rdf_clf, svc_clf, extratree_clf, mlp_clf]
for classifier in classifiers:
  print(classifier,":", classifier.score(X_valid, y_valid))

RandomForestClassifier(random_state=42) : 0.9689
LinearSVC(dual=True, max_iter=100, random_state=42, tol=20) : 0.8768
ExtraTreesClassifier(random_state=42) : 0.9709
MLPClassifier(random_state=42) : 0.9621


SoftVoting, HardVotingを実装

In [ ]:
from sklearn.ensemble import VotingClassifier

estimators = [
    ("rdf_clf", rdf_clf),
    ("svm_clf", svc_clf),
    ("extra_trees_clf", extratree_clf),
    ("mlp_clf", mlp_clf)
]

In [ ]:
voting_clf = VotingClassifier(estimators)
voting_clf.fit(X_train, y_train)
voting_clf.score(X_valid, y_valid)

0.969

下がったぞ．．．
SVMが原因？外してみる．

In [ ]:
voting_clf.set_params(svm_clf = "drop")

VotingClassifier(estimators=[('rdf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('svm_clf', 'drop'),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [ ]:
svm_clf_trained = voting_clf.named_estimators_.pop("svm_clf")
voting_clf.estimators_.remove(svm_clf_trained)

In [ ]:
voting_clf.fit(X_train, y_train)
voting_clf.score(X_valid, y_valid)

0.9715

これで全ての子予測器よりも高いスコアが出た．やっぱりLinearSVCが悪さをしていたみたい．

In [ ]:
voting_clf.voting = "soft"
voting_clf.score(X_valid, y_valid)

0.9678

hard-votingの勝ち.hard-votingを採用してtest-setのスコアを計算．

In [ ]:
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9739

In [ ]:
for estimator in estimators:
  print(estimator[0],":", estimator[1].score(X_test, y_test))

rdf_clf : 0.9704
svm_clf : 0.8717
extra_trees_clf : 0.9715
mlp_clf : 0.9597


OK!

## 7.9

In [ ]:
import numpy as np
X_valid_pred = np.empty((len(X_valid), len(classifiers)), dtype = object)

for index, estimator in enumerate(classifiers):
  X_valid_pred[:,index] = estimator.predict(X_valid)

X_valid_pred

array([['7', '7', '7', '7'],
       ['2', '2', '2', '2'],
       ['1', '1', '1', '1'],
       ...,
       ['4', '4', '4', '4'],
       ['5', '5', '5', '5'],
       ['6', '6', '6', '6']], dtype=object)

In [ ]:
rdf_blender = RandomForestClassifier(random_state = 42, n_estimators = 100, oob_score = True)
rdf_blender.fit(X_valid_pred, y_valid)

RandomForestClassifier(oob_score=True, random_state=42)

In [ ]:
rdf_blender.oob_score_

0.9687

svmを取り除いてみる

In [ ]:
classifiers.remove(svc_clf)

In [ ]:
X_valid_pred = np.empty((len(X_valid), len(classifiers)), dtype = object)

for index, estimator in enumerate(classifiers):
  X_valid_pred[:,index] = estimator.predict(X_valid)

In [ ]:
rdf_blender_rev = RandomForestClassifier(random_state = 42, n_estimators = 100, oob_score = True)
rdf_blender_rev.fit(X_valid_pred, y_valid)
rdf_blender_rev.oob_score_

0.9692

微増．

Test setのスコアを確かめる．

In [ ]:
X_test_pred = np.empty((len(X_test), len(classifiers)), dtype = object)
for index, estimator in enumerate(classifiers):
  X_test_pred[:,index] = estimator.predict(X_test)

X_test_pred

array([['5', '5', '5'],
       ['0', '0', '0'],
       ['4', '4', '4'],
       ...,
       ['6', '6', '6'],
       ['9', '9', '9'],
       ['7', '7', '7']], dtype=object)

In [ ]:
rdf_blender_test = RandomForestClassifier(random_state = 42, n_estimators = 100, oob_score = True)
rdf_blender_test.fit(X_test_pred, y_test)
rdf_blender_test.oob_score_

0.9702

VotingClassifierより悪い．